__1.__ Прочитайте исходный файл с данными. Визуализируйте исходный временной ряд, сделайте первичные выводы о присутствии компонент тренда и сезонности в ряде.

Отложите последние три года из датасета как тестовую выборку для оценки результатов предсказания.


In [ ]:
import pandas as pd

data = pd.read_csv("data/ghana_gdp.csv", parse_dates=["year"], index_col=["year"])
data.rename(columns = {'GDP (current US$)':'gdp'}, inplace = True )
data.index

In [ ]:
data.shape

In [ ]:
# рисуем график 
from matplotlib import pyplot as plt

%matplotlib inline

plt.rcParams["figure.figsize"] = 24, 12
plt.rcParams["font.size"] = 20
plt.rcParams["xtick.labelsize"] = 20
plt.rcParams["ytick.labelsize"] = 20
plt.rcParams["lines.linewidth"] = 4

data.plot()
plt.show()

In [ ]:
# разделяем датасет на тренировочную и тестовую выборки 
train_df = data.loc[:"2018-12-31"] # c начала и до конца 2018
test_df = data.loc["2019-01-01":]  # c 2019 до конца
train_df.shape, test_df.shape

In [ ]:
# декомпозиция графика
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams["figure.figsize"] = 18, 20

result = seasonal_decompose(train_df.gdp)
result.plot()
plt.show()

Выводы: в датасете присутствует явный тренд, без сезонной составляющей и шума.

__2.__ Постройте график скользящего среднего, проанализируйте результат. Для построения MA используйте метод rolling(), который принимает в качестве параметра размер окна. Используйте среднее как функцию для сглаживания. Ширину окна подберите самостоятельно.

In [6]:
def moving_av(data,w=1):
    fig, ax = plt.subplots(nrows= 1 , ncols= 2, figsize = (16,4))
    plt.rcParams["font.size"] = 10
    plt.rcParams["xtick.labelsize"] = 10
    plt.rcParams["ytick.labelsize"] = 10
    plt.rcParams["lines.linewidth"] = 3
    ax[0].plot(data, color="blue")
    ax[0].set_title("original")
    ax[1].plot(
                data.rolling(window=w).mean(),
                color="orange"
               )
    ax[1].set_title("window = " + str(w))    

In [ ]:
for i in range(2,10):
    moving_av(train_df.gdp,w=i)

вывод: __window = 3__ описывает тренд и позволяет избегать выбросов. 

__3.__ помощью теста Дики-Фуллера оцените стационарность временного ряда и примите решение о выборе модели ARMA/ARIMA.


In [8]:
from statsmodels.tsa.stattools import adfuller

def chk_d(data, nbr_dif=0):
    # функция ищет порядок диффенциирования до стационарности ряда или до значения nbr_dif
    # nbr_dif - предельное кол-во раз диффренцирования.
    d=0
    while d<=nbr_dif:
        if adfuller(data)[0] < adfuller(data)[4]["5%"]:
            return print(f"ряд стационарен. порядок диффренцирования 'd' = {d}")
        else:
            print(f"ряд не стационарен. порядок диффренцирования 'd' = {d}")
        data = data.diff().dropna()        
        d+=1

In [ ]:
chk_d(train_df, nbr_dif=3)

Примечание. Если ваш ряд является нестационарным, дифференцируйте его до тех пор, пока он не станет таковым. Количество дифференцирований, необходимых для сведения ряда к стационарному, будет вашим параметром d для модели ARIMA.

вывод: выбираем __ARIMA. d=3__

Параметры p и q выберите по коррелограммам ACF и PACF.
Примечание. Помните, что параметры p и q для ARMA/ARIMA определяются из коррелограмм стационарного ряда. То есть, если ваш изначальный временной ряд не являлся стационарным, то коррелограммы строятся для разностей того порядка, которые являются стационарными.

In [10]:
# дифференцируем до 3-х
data1 = train_df.diff().dropna() # 1
data2 = data1.diff().dropna()    # 2
data3 = data2.diff().dropna()    # 3

In [ ]:
data3.head(2)

In [ ]:
import statsmodels.graphics.tsaplots as sgt

plt.rcParams["figure.figsize"] = 20, 5
sgt.plot_acf(data3.gdp, lags=20, zero=False)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = 20, 5
sgt.plot_pacf(data3.gdp, lags=20, method="ywm", zero=False)
plt.show()

__p = 6(по графику pacf) , q = 1(по графику acf)__

Начнем построение модели **ARIMA** со следующими параметрами:

- p = 6 
- d = 3 
- q = 1 

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

arima_model = ARIMA(train_df["gdp"].dropna(), order=(6, 3, 1))
arima_model_fit = arima_model.fit()
arima_model_fit.summary()

Также постройте несколько моделей с параметрами, ближайшими к найденным p и q, и сравните коэффициент AIC (или подберите наилучшие параметры с помощью пакета pmdarima, как в скринкасте).

In [ ]:
import pmdarima as pm

smodel = pm.auto_arima(
    train_df,
    start_p=1,
    d=3,
    start_q=1,
    max_p=12,
    max_q=12,
    test="adf",
    error_action="ignore",
    trace=True,
)

smodel.summary()

Вывод: параметры подобранный вручную по критерию AIC оказались лучше. 

__4.__ Постройте модель с наилучшими параметрами и выполните предсказание для отложенной тестовой выборки (последние три года).

In [16]:
pred_df = pd.DataFrame(index=data.index)

# предсказание для тестовой выборки
pred_df["model_preds"] = arima_model_fit.predict(
    start=test_df.index[0], end=test_df.index[-1]
)

pred_df["gdp"] = data["gdp"]

forecast_arima = arima_model_fit.get_forecast(len(test_df.index))
forecast_df = forecast_arima.conf_int(alpha=0.05)  # Confidence level of 95%

In [17]:
pred_df.at[test_df.index, "model_preds_lower"] = forecast_df["lower gdp"]
pred_df.at[test_df.index, "model_preds_upper"] = forecast_df["upper gdp"]

__5.__ Отобразите результат графически — постройте графики истинного и предсказанного поведения временного ряда, а также 95%-ый доверительный интервал для прогноза.

In [ ]:
plt.rcParams["figure.figsize"] = 18, 12

plt.subplot(2, 1, 1)
plt.plot(pred_df["gdp"], color="blue", label="Actuals", alpha=0.4)

plt.plot(
    pred_df.loc[test_df.index]["model_preds"],
    color="red",
    linestyle="-",
    label="Out of Sample Fit",
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_lower"],
    color="green",
    linestyle="--",
    label="Confidence Intervals (95%)",
    alpha=0.4,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_upper"],
    color="green",
    linestyle="--",
    alpha=0.4,
)

plt.title("Full Model Fit", size=24)
plt.legend()

plt.subplot(2, 2, 3)

plt.plot(
    pred_df.loc[test_df.index]["gdp"], color="blue", label="Actuals", alpha=0.6
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds"],
    color="red",
    linestyle="-",
    label="Out of Sample Fit",
    alpha=0.6,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_lower"],
    color="green",
    linestyle="--",
    label="Confidence Intervals (95%)",
    alpha=0.6,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_upper"],
    color="green",
    linestyle="--",
    alpha=0.6,
)

plt.title("Out of Sample Fit", size=24)
plt.legend()

plt.show()

__6.__ Сделайте выводы по полученным результатам.

__*__ на части тестовых данных модель не уловила тренд

__*__ по метрикам модель с подобранными вручную параметрами показали лучший результат

__7.__ Проверьте данные на наличие пропущенных дат. Помимо визуального способа, это можно сделать с помощью метода DataFrame.asfreq(), передав в него параметр частоты, например ‘d’ — день, ‘m’ — месяц. Все алиасы для параметров частоты доступны по ссылке.

In [ ]:
data = pd.read_csv("data/ghana_gdp.csv", parse_dates=["year"], index_col=["year"])
data.rename(columns = {'GDP (current US$)':'gdp'}, inplace = True )
data.index

In [ ]:
print('размер до обработки: {}.'.format(data.shape[0]))
data = data.asfreq(freq='AS')
print('размер после обработки: {}.'.format(data.shape[0]))

In [21]:
# заменяем нули на NaN
import numpy as np
data['gdp'] = data['gdp'].apply(lambda x: np.nan if x==0 else x)

In [ ]:
# имеем одну NaN строку
data['gdp'].isnull().sum()

In [23]:
# заменяем NaN значение на интерполированное
data['gdp'].interpolate(method='linear', inplace=True)

+ Проверьте полученный ряд на стационарность, определите параметры модели (ARIMA/ARMA) и запустите модель.
+ Изменился ли AIC критерий построенной модели по сравнению с моделью на неинтерполированных данных? Сделайте вывод.

In [ ]:
# проверяем на стационарность и определим параметр d (d=3)
chk_d(data, nbr_dif=4)

In [25]:
# проверим другие параметры модели по кореллограмам
data1 = train_df.diff().dropna() # 1
data2 = data1.diff().dropna()    # 2
data3 = data2.diff().dropna()    # 3

In [ ]:
plt.rcParams["figure.figsize"] = 20, 5
sgt.plot_acf(data3.gdp, lags=20, zero=False)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = 20, 5
sgt.plot_pacf(data3.gdp, lags=20, method="ywm", zero=False)
plt.show()

__p = 6(по графику pacf) , q = 1(по графику acf)__ параметры не изменились

In [ ]:
# запускаем ARIMA
train_df = data.iloc[:-3]
test_df = data.iloc[-3:]
arima_model = ARIMA(train_df["gdp"].dropna(), order=(6, 3, 1))
arima_model_fit = arima_model.fit()
arima_model_fit.summary()

In [ ]:
plt.rcParams["figure.figsize"] = 18, 12

plt.subplot(2, 1, 1)
plt.plot(pred_df["gdp"], color="blue", label="Actuals", alpha=0.4)

plt.plot(
    pred_df.loc[test_df.index]["model_preds"],
    color="red",
    linestyle="-",
    label="Out of Sample Fit",
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_lower"],
    color="green",
    linestyle="--",
    label="Confidence Intervals (95%)",
    alpha=0.4,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_upper"],
    color="green",
    linestyle="--",
    alpha=0.4,
)

plt.title("Full Model Fit", size=24)
plt.legend()

plt.subplot(2, 2, 3)

plt.plot(
    pred_df.loc[test_df.index]["gdp"], color="blue", label="Actuals", alpha=0.6
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds"],
    color="red",
    linestyle="-",
    label="Out of Sample Fit",
    alpha=0.6,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_lower"],
    color="green",
    linestyle="--",
    label="Confidence Intervals (95%)",
    alpha=0.6,
)

plt.plot(
    pred_df.loc[test_df.index]["model_preds_upper"],
    color="green",
    linestyle="--",
    alpha=0.6,
)

plt.title("Out of Sample Fit", size=24)
plt.legend()

plt.show()

+ Показатель AIC слегка улучшился(2628.95 против 2629.140). Но явно недостаточно, так как поменялось одно значение в данных

+ Посчитайте волатильность для вашего временного ряда.
+ Используйте GARCH-модель для предсказания волатильности.
+ Используйте линейную регрессию для получения прогноза.
+ Визуализируйте полученное предсказание вместе с фактическим значением.
+ Сравните результаты и сделайте выводы.
 Почему результат модели GARCH получился именно таким?

In [ ]:
data["gdp_volat"] = data['gdp'].pct_change(1).mul(100).abs()
data["gdp_volat"].plot()
plt.show()

In [32]:
train_df = data.iloc[:-3]
test_df = data.iloc[-3:]

In [ ]:
m_value = train_df.gdp_volat.median()
train_df.fillna(value=m_value, inplace=True )

In [ ]:
train_df.head()

In [ ]:
chk_d(train_df.gdp_volat, nbr_dif=3)

In [ ]:
# выбираем параметр p = 7
sgt.plot_pacf(train_df.gdp_volat, lags=20, method="ywm")
plt.show()

In [37]:
# нам важно сохранить объем датасета, поэтому берем за исходные данные весь датасет с индексами, который содержит
# и train и test часть
garch_df = pd.DataFrame(data["gdp_volat"].loc[data.index])

In [ ]:
#! pip install arch
from arch import arch_model
model = arch_model(garch_df["gdp_volat"].dropna(), p=7, q=0, vol='GARCH')
model_results = model.fit(last_obs=test_df.index[0], update_freq=5)
model_results.summary()

In [ ]:
test_df["Predictions"] = model_results.forecast().residual_variance.loc[test_df.index]
test_df.head()

In [ ]:
plt.rcParams["figure.figsize"] = 18, 5

plt.plot(test_df["gdp_volat"].abs(), color = 'Green', label = 'Real')
plt.plot(test_df["Predictions"] , color = 'Red', label = 'Prediction')

plt.title("Actuals vs Predictions for GDP Volatility", size=24)
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

print('MSE score: {:.3f} '.format((mean_squared_error(test_df['gdp_volat'], 
                                                                     test_df['Predictions']))))

In [ ]:
from sklearn.linear_model import LinearRegression

X_train = pd.DataFrame(range(data["gdp"].shape[0]))[
    : train_df.shape[0]
].fillna(0)
X_test = pd.DataFrame(range(data["gdp"].shape[0]))[
    train_df.shape[0] :
].fillna(0)
y_train = train_df["gdp"].fillna(0)
y_test = test_df["gdp"].fillna(0)


lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)


plt.plot(y_test, color = 'Green', label='Real')
plt.plot(pd.Series(lr_predict, index=y_test.index), color='Red', label='Predict')
plt.legend()
plt.show()

Вывод: недостаток обучающих данных не позволил обучить модель которая бы отработала дисперсию. Линейная модель с параметрами по умолчанию также не сумела построить нормальную модель.